# Сравнение методов линейного поиска, backtracking method

Про реализацию и теорОбоснование `Backtracking method` — см. в первых частях.

## Что хотим от line search метода

Мы уже собрали целый зоопарк методов линейного поиска:
- БинПоиск по градиенту (с остановкой по достижению малого значения градиента и/или размера ворот $r - l$, с опциональным обрубанием по количеству итераций)
- Золотое сечение (по количеству итераций)
- Фибоначчи (по количеству итераций)
- Метод, базирующийся на условиях Вольфе (aka backtracking method)

Здесь нужно проанализировать, какие `line searcher`-ы _лучше_ себя показывают. Что от такого алгоритма требуется?

1. Конечно же, хорошо сходиться
2. Произвести _достаточно_ точный поиск, чтобы заЭксплойтить это направление поиска
3. Но не переусердствовать: не слишком долго искать, чтобы сэкономить время

Пункт 1 очевиден и не настолько интересен.
А вот между 2. и 3. налицо трейд-офф по поводу того, насколько быстро надо останавливаться. Для каждой конкретной функции ответ на этот вопрос разный — и хороший line search метод — тот, который хорошо умеет его давать.
Причём давать для самых разных паттернов ландшафта, масштабирований и т.д.

## Как производить оценку

Раз уж так важна устойчивость и догадливость метода, заведём набор _интересных_ функций и попробуем запустить методы на них (метод проходит все функции с одинаковыми гиперпараметрами — приближено к реальности).

Функции:
- квадратичная (какое-то/какие-то число(а) обусловленности, размерность(и))
- умножение на разные константы (маленькую, большую)/масштабирование аргументов
- непредсказуемый рельеф (всё ещё выпуклая, но где-то сильно убывает, где-то — _не очень_…)

Теперь — как оценивать результат. В целом — чем меньше действий, тем лучше. В реальной жизни обычно речь идёт о процессорном времени, но здесь измерять время — это лишняя возня, потеря точности и, главное, bias за счёт того, что у тестовых функций (к примеру, квадратичной) может быть _странное_ отношение времён вычисления функции и градиента. Это затрудняет оценку самого метода линейного поиска, а также не соответствует реальности. Поэтому будем считать, что целевая функция (чем меньше, тем лучше) в данном случае $\operatorname{computations}(f) + \operatorname{computations}(\nabla f)$.

Ожидаем, что методы с фиксированным количеством итераций будут хорошо себя проявлять только для конкретных функций фиксированного уровня сложности, а для других — либо делать избыточную работу внутри каждой итерации, либо плохо отрабатывать очередное направление → потребуется много внешних итераций, иногда даже вообще не сходиться.

А про методы с остановкой по фиксированному $\varepsilon$ ожидаем, что он будет плохо устойчив к масштабированию функции/градиента.